In [1]:
import os
import random
from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import tensorflow as tf
from google_drive_downloader import GoogleDriveDownloader as gdd
from sklearn.model_selection import train_test_split

tqdm.pandas()

np.set_printoptions(precision=3)

C:\Users\5CG7092POZ\Anaconda3\envs\tf2.0\lib\site-packages\tqdm\std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
gdd.download_file_from_google_drive(file_id="19YPTvLXn6xIbqtRi9_6q6yCZdmopWjGE", dest_path="./100k_users.csv")
users_df = pd.read_csv("100k_users.csv")

gdd.download_file_from_google_drive(file_id="1oVBV039q3aXC16O05Eka4dggcEvX4csA", dest_path="./100k_movies.csv")
movies_df = pd.read_csv("100k_movies.csv")

gdd.download_file_from_google_drive(file_id="12ypQN2hAESdEXNE64GdCtvZlz5ijI4o6", dest_path="./100k_ratings.csv")
ratings_df = pd.read_csv("100k_ratings.csv")

In [3]:
users_df.sample()

,user_id,age,gender,occupation
793,794,6,M,educator


In [4]:
movies_df.sample()

,item_id,title,year,unknown,Action,Adventure,Animation,Children,Comedy,Crime,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
737,739,Pretty Woman (1990),1990,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0


In [5]:
ratings_df.sample()

,user_id,item_id,rating
43267,329,879,2


## K-Core Sampling을 이용해 사용자와 영화 선별하기
- k=thrsd 이상의 평점을 남김 사용자와 k=thrsd 이상의 평점이 남겨진 영화를 제외하고 나머지를 제거합니다.

In [6]:
thrsd = 5
cnt = 0
while True:
    cnt += 1
    len_before = len(ratings_df)
    print(f"len(ratings_df) : {len(ratings_df):,}")
    #thrsd 이상의 영화에 평점을 남긴 사용자들의 리스트
    n_ratings_per_user = ratings_df["user_id"].value_counts()
    exec(f"users_over{thrsd}ratings = n_ratings_per_user[n_ratings_per_user>thrsd].index")

    #thrsd 이상의 평점이 남겨진 영화들의 리스트
    n_users_per_movie = ratings_df["item_id"].value_counts()
    exec(f"movies_over{thrsd}ratings = n_users_per_movie[n_users_per_movie>thrsd].index")

    ratings_df = ratings_df[(ratings_df["user_id"].isin(eval(f"users_over{thrsd}ratings")) & ratings_df["item_id"].isin(eval(f"movies_over{thrsd}ratings")))]
    len_after = len(ratings_df)
    #더 이상 갯수의 변화가 없으면 종료합니다.
    if len_after == len_before:
        break
print("finished!")

len(ratings_df) : 99,991
len(ratings_df) : 99,023
finished!


### 4. 평가 기준 설정하기
Bayesian Personalized Ranking과 Neural Collaborative Filtering의 성능을 비교해보기 위해 우리는 Hit Ratio를 이용하도록 하겠습니다.

### (1) 평가데이터셋 구성하기
우선 Train 데이터셋와 Test 데이터셋을 나누도록 하겠습니다. Test 데이터셋은 각 고객이 평가한 마지막 영화로 두도록 하겠습니다.

In [7]:
ratings_df

,user_id,item_id,rating
0,196,242,3
1,186,302,3
2,22,377,1
3,244,51,2
4,166,346,1
...,...,...,...
99986,880,476,3
99987,716,204,5
99988,276,1090,1
99989,13,225,2


In [8]:
ratings_df_train = pd.DataFrame()
ratings_df_test = pd.DataFrame()
for idx, group in tqdm(ratings_df.groupby(["user_id"])):
    train, test = train_test_split(group, test_size=0.1, shuffle=False)
    ratings_df_train = pd.concat([ratings_df_train, train], axis=0)
    ratings_df_test = pd.concat([ratings_df_test, test], axis=0)

In [9]:
print(f"len(ratings_df_train) : {len(ratings_df_train):>10,}")
print(f"len(ratings_df_test)  : {len(ratings_df_test):>10,}")

len(ratings_df_train) :     88,681
len(ratings_df_test)  :     10,342


### 2) Hit Ratio를 위한 데이터셋 구성하기
- 이번 시간에는 평가지표를 Hit Ratio를 이용해보도록 하겠습니다.

Hit Ratio의 측정방법은 아래와 같습니다.

고객이 구매한 아이템 1개와 고객이 구매하지 않은 아이템 100개를 가져온 후, 고객이 구매한 아이템 고객이 구매한 아이템이 101개 중 몇번째에 위치하는지를 확인하기. Top-10 Hit Ratio란, 고객이 구매한 아이템이 10번째 안에 들어있는 확률로, 높을수록 보다 정확하게 추천한다고 판단

In [10]:
seen_per_user = ratings_df.groupby(["user_id"])["item_id"].apply(frozenset)

In [11]:
seen_per_user.sample(5)

user_id
795    (1, 514, 3, 4, 2, 1030, 7, 8, 10, 1036, 12, 10...
29     (259, 264, 268, 269, 270, 12, 657, 661, 539, 2...
158    (1, 514, 516, 4, 518, 7, 8, 10, 11, 525, 530, ...
828    (512, 6, 10, 14, 19, 20, 531, 24, 26, 1056, 54...
35     (1025, 258, 259, 261, 264, 266, 678, 680, 937,...
Name: item_id, dtype: object

- hit ratio를 계산하기 위해서 우리는 본 영화 1개와 보지 않은 영화 100개를 구성해야 합니다. 그리고 본 영화와 보지 않은 영화 모두 모델로 추론한 후, 선호도 순서대로 정렬 후 본 영화가 10등 안에 들었으면, 모델이 올바르게 추론했다고 평가하고, 들지 않으면 모델이 잘못 추론했다고 평가하는 방식입니다.

In [12]:
#영화별 평점의 수를 나타낸 series를 만듭니다.
n_per_movie = ratings_df.groupby(["item_id"]).size().sort_values(ascending=False)

In [20]:
n_per_movie

item_id
50      583
258     509
100     508
181     507
294     485
       ... 
1412      6
296       6
1083      6
867       6
438       6
Length: 1297, dtype: int64

In [14]:
hit_ratio_df = ratings_df_test.copy()

#user_id에 따라 해당 사용자가 본 영화들의 집합을 column seen에 추가합니다.
hit_ratio_df = hit_ratio_df.drop(["rating"], axis=1)
hit_ratio_df["seen"] = hit_ratio_df["user_id"].progress_apply(lambda x : seen_per_user[x])

In [15]:
def pick_unseen_100(x):
    n_per_movie_unseen = n_per_movie[~n_per_movie.index.isin(x["seen"])]
    return set(n_per_movie_unseen.sample(100, replace=False, weights=n_per_movie).index)

hit_ratio_df["unseen_100"] = hit_ratio_df.progress_apply(lambda x : pick_unseen_100(x), axis=1)

In [16]:
hit_ratio_df

,user_id,item_id,seen,unseen_100
77066,1,124,"(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","{515, 517, 523, 1037, 527, 531, 1078, 582, 584..."
77231,1,95,"(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","{1051, 540, 546, 550, 557, 565, 568, 1598, 574..."
77623,1,217,"(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","{515, 1028, 1032, 521, 1033, 527, 1046, 1047, ..."
78164,1,58,"(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","{1025, 514, 515, 1028, 519, 525, 529, 533, 106..."
78688,1,142,"(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","{515, 520, 1039, 530, 1046, 1051, 1059, 552, 5..."
...,...,...,...,...
99163,943,415,"(2, 1028, 9, 11, 12, 526, 1044, 22, 23, 24, 10...","{1, 3, 8, 13, 14, 15, 527, 1041, 1049, 1051, 3..."
99311,943,219,"(2, 1028, 9, 11, 12, 526, 1044, 22, 23, 24, 10...","{512, 513, 1, 3, 4, 517, 8, 14, 1039, 531, 104..."
99603,943,796,"(2, 1028, 9, 11, 12, 526, 1044, 22, 23, 24, 10...","{514, 4, 524, 13, 527, 529, 530, 535, 25, 537,..."
99823,943,739,"(2, 1028, 9, 11, 12, 526, 1044, 22, 23, 24, 10...","{512, 7, 523, 527, 15, 530, 1060, 551, 564, 61..."


## TensorFlow로 Bayesian Personalized Ranking 구현하기
- BPR의 핵심 아이디어는 바로 고객이 본 영화에 대한 선호도가 보지 않은 영화에 대한 선호도보다 항상 높다는 것입니다.
- 고객, 고객이 본 영화, 고객이 보지 않은 영화 이렇게 3개의 입력이 들어갑니다.

### 1. 모델 구성하기

#### (1) Input 구성하기

In [29]:
user_id = tf.keras.Input(shape=(), name="user")
pos_item_id = tf.keras.Input(shape=(), name="positive_items")
neg_item_id = tf.keras.Input(shape=(), name="negative_items")

#### (2) 임베딩 레이어 구성하기
- Item Embedding의 경우, 각 아이템 별 편향 정보(Bias)를 추가하기 위해 Num Factor에 1을 더하도록 하겠습니다.

In [28]:
ratings_df

,user_id,item_id,rating
0,196,242,3
1,186,302,3
2,22,377,1
3,244,51,2
4,166,346,1
...,...,...,...
99986,880,476,3
99987,716,204,5
99988,276,1090,1
99989,13,225,2


In [53]:
n_users = len(set(ratings_df["user_id"]))
n_items = len(set(ratings_df["item_id"]))
n_facts = 30

user_emb_layer = tf.keras.layers.Embedding(n_users, n_facts+1, name="user_embedding_layesr")
item_emb_layer = tf.keras.layers.Embedding(n_items, n_facts+1, name="item_embedding_layers")

### (3) Item Embedding, User Embedding 구하기
Tensorflow Keras에서 Item Embedding의 값과 User Embedding의 값을 가져오는 것은 매우 간단합니다. 우리는 층의 연결을 통해 가져올 수 있습니다.

#### Item Embedding 구하기
이 때 주의해야 하는 것은 positive item과 negative item은 같은 임베딩 레이어에서 가져와야 합니다.

In [84]:
pos_item_emb = item_emb_layer(pos_item_id)
neg_item_emb = item_emb_layer(neg_item_id)

#### User Embedding 구하기¶
유저 임베딩에서 우리는 마지막 임베딩에 1을 추가해주어야 합니다. 아이템 임베딩의 마지막 원소값 Bias를 추가하기 위함입니다.바로 아래와 같은 방식으로 유저 임베딩과 아이템 임베딩이 형성됩니다.

U=[u1,u2,u3,...,u60,1]I=[i1,i2,i3,...,i60,ibias]
Dot 연산으로 Bias 연산까지 같이 수행하기 위해 아래와 같이 코드를 작성하게 됩니다.

In [88]:
user_emb = user_emb_layer(user_id)
# one_emb = tf.ones_like(user_emb[:, -1:])

### (4) Score 계산하기
- 우리는 고객이 본 아이템에 대한 Score와 고객이 보지 않은 아이템에 대한 Score의 차이가 극대화되도록 학습하게 됩니다. 이를 위해 BPR에서는 Individual Probability, 즉 고객이 본 아이템에 대해 보지 않은 아이템보다 선호할 확률을 구하게 됩니다.
- Bayesian Personalized Ranking은 위의 확률이 100%가 되도록 학습합니다.

In [97]:
pos_score = tf.keras.layers.Dot(axes=(1, 1))([user_emb, pos_item_emb])
neg_score = tf.keras.layers.Dot(axes=(1, 1))([user_emb, neg_item_emb])

diff = pos_score - neg_score

probs = tf.keras.activations.sigmoid(diff)

In [98]:
probs

<tf.Tensor 'Sigmoid_1:0' shape=(None, 1) dtype=float32>

### (5) Model 구성하기
입력값은 크게 세가지 user_id, pos_item_id,neg_item_id으로 나뉘어집니다. 그리고 출력값은 보지 않은 아이템에 대한 선호도보다 본 아이템에 대한 선호도가 높을 확률(individual probability)인 probs이 됩니다.

In [99]:
model = tf.keras.Model([user_id, pos_item_id, neg_item_id], probs)

### (6) Regularization 적용하기
- Matrix Factoriation은 쉽게 Overfitting, 즉 학습 데이터에만 과적합되는 현상이 발생합니다. 이를 방지하기 위해 가장 기본적인 방법론 중 하나는 Weight Decay, 즉 weight의 값이 너무 커지지 않도록 방지하는 것입니다. 이를 위해 아래와 같이 Loss를 추가해주게 되면, weight가 어느정도 줄어드는 방향으로 모델이 학습하게 됩니다.

In [100]:
l2_pos_item = pos_item_emb**2
l2_neg_item = neg_item_emb**2
l2_user = user_emb**2
l2_reg = 0.0001

weight_decay = l2_reg * tf.reduce_sum(l2_pos_item + l2_neg_item + l2_user)

model.add_loss(weight_decay)

### (7) Model 컴파일하기
이전 구현체인 implicit에서는 기본 SGD를 이용했지만, 여기에서는 최대한 빠르게 수렴시키기 위해 변형체인 Adagrad를 이용하도록 하겠습니다.

In [101]:
model.compile(tf.keras.optimizers.Adagrad(1), loss=tf.keras.losses.BinaryCrossentropy, metrics=[tf.keras.metrics.BinaryAccuracy()])

TypeError: Failed to convert object of type <class 'tensorflow.python.keras.losses.BinaryCrossentropy'> to Tensor. Contents: <tensorflow.python.keras.losses.BinaryCrossentropy object at 0x00000239F29E0508>. Consider casting elements to a supported type.

## 2. 학습 데이터 구성하기
- 우리가 가지고 있는 데이터는 고객이 특정 영화에 대해 몇점의 평점을 주었는지에 대한 데이터입니다. 우리는 고객이 평가하지 않은 영화에 대한 정보를 생성해서 Pair 단위로 모델을 학습해야 합니다.

### 1) 고객이 평가하지 않은, 구매하지 않은 영화군 정의하기

In [131]:
seen_per_user = ratings_df_train.groupby(["user_id"])["item_id"].apply(frozenset)

all_movies = set(ratings_df_train["item_id"])

unseen_per_user = all_movies - seen_per_user
unseen_per_user = unseen_per_user.apply(list)

### (2) 학습 데이터 구성하기
우리는 매 Epoch마다 본것과 보지 않은 것에 대한 쌍을 무작위로 추출합니다. 그리고 학습 데이터에서 출력값은 항상 1로 나와야 합니다.(보지 않은 것에 대한 본것의 확률 = 100%)

In [132]:
def get_bpr_dataset(ratings_df_train, unseen_per_user):
    ratings_df_train_batch = ratings_df_train.copy()
    ratings_df_train_batch = ratings_df_train_batch.sample(frac=1)
    ratings_df_train_batch["neg_item"] = ratings_df_train_batch.apply(lambda x : random.choice(unseen_per_user.loc[x["user_id"]]), axis=1)
    
    x = {"user":ratings_df_train_batch["user_id"].values, "positive_items":ratings_df_train_batch["item_id"].values, "negative_items":ratings_df_train_batch["neg_item"].values}
    
    y = np.ones((len(ratings_df_train_batch), 1))
    
    return x, y

### (3) 모델 학습하기
- epoch 10번에 걸쳐 모델을 학습시키도록 하겠습니다. 매 Epoch마다 새로운 학습 pair를 생성하도록 하였습니다.

In [136]:
ratings_df_train_batch = ratings_df_train.copy()
ratings_df_train_batch = ratings_df_train_batch.sample(frac=1)
ratings_df_train_batch["neg_item"] = ratings_df_train_batch.apply(lambda x : random.choice(unseen_per_user.loc[x["user_id"]]), axis=1)

In [137]:
ratings_df_train_batch

,user_id,item_id,rating,neg_item
30587,532,586,4,454
85244,878,276,3,1166
63516,848,95,5,761
50401,176,319,3,983
65308,567,481,5,108
...,...,...,...,...
34620,484,135,4,146
74411,479,474,5,245
13457,130,99,5,205
72232,847,216,3,1313


In [133]:
n_epochs = 10
for i in range(1, n_epochs+1):
    print(f"epoch : {i:>3d}")
    x, y = get_bpr_dataset(ratings_df_train, unseen_per_user)
    model.fit(x, y, batch_size=64, verbose=2)

epoch :   1
Train on 88681 samples


TypeError: Failed to convert object of type <class 'tensorflow.python.keras.losses.BinaryCrossentropy'> to Tensor. Contents: <tensorflow.python.keras.losses.BinaryCrossentropy object at 0x00000239F8957908>. Consider casting elements to a supported type.